In [138]:
from tqdm import tqdm

# bag of wordsの辞書を作成
def create_BoW_dict(file_path):
    BoW_dict = {}
    i = 0
    with open(file_path, "r", encoding="utf-8")as f:
        data_list = f.readlines()
        for data in data_list:
            data = data.split('\t')
            word_list = data[0].split()
            for word in word_list:
                if word in BoW_dict:
                    continue
                else:
                    BoW_dict[word] = i
                    i += 1
    return BoW_dict

# t=0, e=1, m=2, b=3としてテストデータのベクトルとラベルをそれぞれリストとして作成
def create_vector_and_label(file_path, BoW_dict):
    with open(file_path, "r", encoding="utf-8")as f:
        data_list = f.readlines()
        vector_list = []
        label_list = []
        for data in tqdm(data_list):
            data = data.split("\t")
            word_list = data[0].split()
            vector = [0] * len(BoW_dict)
            for word in word_list:
                if word in BoW_dict:
                    vector[BoW_dict[word]] += 1
            vector_list.append(vector)
            label_alph = data[1].strip("\n")
            if label_alph == "t":
                label = [1, 0, 0, 0]
            elif label_alph == "e":
                label = [0, 1, 0, 0]
            elif label_alph == "m":
                label = [0, 0, 1, 0]
            elif label_alph == "b":
                label = [0, 0, 0, 1]
            else:
                print("error label")
            label_list.append(label)
    return vector_list, label_list

In [139]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

TRAIN_FEATURE_FILE_PATH = "./news+aggregator/train.feature.txt"

# bag of wordsの辞書型を作成
BoW_dict = create_BoW_dict(TRAIN_FEATURE_FILE_PATH)

# test.txtからベクトルリストとラベルリストを作成
x_train_list, y_train_list = create_vector_and_label(TRAIN_FEATURE_FILE_PATH, BoW_dict)

# データの準備
x_train = torch.Tensor(x_train_list)
y_train = torch.Tensor(y_train_list)

# 訓練データのデータローダー
train = TensorDataset(x_train, y_train)
train_loader = DataLoader(train, batch_size=10, shuffle=True)

100%|██████████| 10672/10672 [00:01<00:00, 8508.30it/s]


In [140]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.l1 = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.l1(x)
        return x

input_dim = len(x_train_list[0])
print(input_dim)
output_dim = 4
model = LogisticRegression(input_dim, output_dim)

19433


In [141]:
lr_late = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr_late)

In [142]:
loss_history = []
for epoch in tqdm(range(5000)):
    total_loss = 0
    for x, y in train_loader:

        # 学習ステップ
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    loss_history.append(total_loss)
    if (epoch +1) % 100 == 0:
        print(epoch + 1, total_loss)


  2%|▏         | 100/5000 [00:33<28:18,  2.88it/s]

100 761.1860098838806


  4%|▍         | 200/5000 [01:08<27:56,  2.86it/s]

200 625.5535930544138


  6%|▌         | 300/5000 [01:45<29:59,  2.61it/s]

...

4800 95.92304863967001


 98%|█████████▊| 4900/5000 [47:04<00:42,  2.38it/s]

4900 94.20696676149964


100%|██████████| 5000/5000 [47:45<00:00,  1.74it/s]

5000 92.46189489495009


In [151]:
torch.save(model, 'model_weight.pth')

In [150]:
correct = 0
total = 0

TEST_FEATURE_FILE_PATH = "./news+aggregator/test.feature.txt"

# test.txtからベクトルリストとラベルリストを作成
x_test_list, y_test_list = create_vector_and_label(TEST_FEATURE_FILE_PATH, BoW_dict)

# データの準備
x_test = torch.Tensor(x_test_list)
y_test = torch.Tensor(y_test_list).long()

# 訓練データのデータローダー
test = TensorDataset(x_test, y_test)
test_loader = DataLoader(test, batch_size=5)

for x, y in test_loader:
    outputs = model(x)
    _, predicted = torch.max(outputs.data, 0)   #確率の最大値を取得
    total += y.size(0)  #batch処理をしているのでy.size == 5
    correct += (predicted == y).sum().item()    #ラベルと一致している個数を計算．　item()はtensor->intへの変換
print('正解率', int(correct)/total*100)

100%|██████████| 1334/1334 [00:00<00:00, 4988.87it/s]


正解率 79.83508245877061
